### A modificar!

Rotació de les imatges: url/label/rotación

## Cargar el dataset
1. Cargar el dataset que son imagenes que están dentro de carpetas separadas por test y train y cada una separada por maligno y benigno.
2. Crearemos un dataset en el que por cada fila se incluye el path y si es benigno o maligno.
3. Tenemos 4000 imagenes para el train y 2000 para el test.

### Dataframe train
Obtener todas las imágenes dentro de la carpeta de train y guardar los Path junto con los nombres de las imagenes y si es maligna o benigna en un Dataframe.

In [107]:
import os
import pandas as pd
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

# Train
def train_df(train_path):
    classes = [] # Benignas o malignas
    class_paths = [] # Paths de las imagenes

    # Obtener el contenido de la carpeta Train (Benigna y Maligna)
    files = os.listdir(train_path)
    for file in files: 
        label_dir = os.path.join(train_path, file)
        label = os.listdir(label_dir)

        # Obtener el contenido de la carpeta Benigna y Maligna (Imagenes)
        for image in label:
            if not image.startswith('.'): # Descartar ficheros ocultos
                class_paths.append(label_dir + '/' + image) 
                classes.append(file) 

    # Crear el dataframe
    class_paths = pd.Series(class_paths, name='Class Path')
    image_classes = pd.Series(classes, name='Class') 
    tr_df = pd.concat([class_paths, image_classes], axis=1)
    
    return tr_df

tr_df = train_df('./melanoma_cancer_dataset/train')
print(tr_df)
groupped = tr_df.groupby(["Class"]).count()
print(groupped)

                                             Class Path      Class
0     ./melanoma_cancer_dataset/train/malignant/mela...  malignant
1     ./melanoma_cancer_dataset/train/malignant/mela...  malignant
2     ./melanoma_cancer_dataset/train/malignant/mela...  malignant
3     ./melanoma_cancer_dataset/train/malignant/mela...  malignant
4     ./melanoma_cancer_dataset/train/malignant/mela...  malignant
...                                                 ...        ...
9600  ./melanoma_cancer_dataset/train/benign/melanom...     benign
9601  ./melanoma_cancer_dataset/train/benign/melanom...     benign
9602  ./melanoma_cancer_dataset/train/benign/melanom...     benign
9603  ./melanoma_cancer_dataset/train/benign/melanom...     benign
9604  ./melanoma_cancer_dataset/train/benign/melanom...     benign

[9605 rows x 2 columns]
           Class Path
Class                
benign           5000
malignant        4605


### Dataframe test
Obtener todas las imágenes dentro de la carpeta de test y guardar los Path junto con los nombres de las imagenes y si es maligna o benigna en un Dataframe.

In [108]:
# Test
def test_df(test_path):
    classes = [] # Benignas o malignas
    class_paths = [] # Ruta a las imagenes

    files = os.listdir(test_path) # Archivos
    for file in files: 
        label_dir = os.path.join(test_path, file) # Path del directorio
        label = os.listdir(label_dir) # Imagenes dentro del directorio
        for image in label:
            if not image.startswith('.'):
                class_paths.append(label_dir + '/' + image) 
                classes.append(file) # Añade el nombre del archivo

    # Series de pandas
    class_paths = pd.Series(class_paths, name='Class Path')
    image_classes = pd.Series(classes, name='Class') 

    # Crea el dataframe
    ts_df = pd.concat([class_paths, image_classes], axis=1)
    return ts_df

ts_df = test_df('./melanoma_cancer_dataset/test')
print(len(ts_df))

1000


### Normalizar los datos entre 0 y 1
Normalizar los datos de Benigno y Maligno de string a numeros (0 o 1)

In [109]:
tr_df['Class'].replace({'benign': 0, 'malignant': 1}, inplace=True)
ts_df['Class'].replace({'benign': 0, 'malignant': 1}, inplace=True) 
print(tr_df, ts_df)

                                             Class Path  Class
0     ./melanoma_cancer_dataset/train/malignant/mela...      1
1     ./melanoma_cancer_dataset/train/malignant/mela...      1
2     ./melanoma_cancer_dataset/train/malignant/mela...      1
3     ./melanoma_cancer_dataset/train/malignant/mela...      1
4     ./melanoma_cancer_dataset/train/malignant/mela...      1
...                                                 ...    ...
9600  ./melanoma_cancer_dataset/train/benign/melanom...      0
9601  ./melanoma_cancer_dataset/train/benign/melanom...      0
9602  ./melanoma_cancer_dataset/train/benign/melanom...      0
9603  ./melanoma_cancer_dataset/train/benign/melanom...      0
9604  ./melanoma_cancer_dataset/train/benign/melanom...      0

[9605 rows x 2 columns]                                             Class Path  Class
0    ./melanoma_cancer_dataset/test/malignant/melan...      1
1    ./melanoma_cancer_dataset/test/malignant/melan...      1
2    ./melanoma_cancer_dataset/te

/var/folders/l4/zsmfmzjn78l0w5yh426c9cn00000gn/T/ipykernel_17955/2617516243.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  tr_df['Class'].replace({'benign': 0, 'malignant': 1}, inplace=True)
/var/folders/l4/zsmfmzjn78l0w5yh426c9cn00000gn/T/ipykernel_17955/2617516243.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent

## Diferenciar los datos
### Partir las X Y
Indicar al Dataframe train y test cuales son los datos y cuales son los resultados

In [110]:
# Train
trainX = tr_df.values[:, :-1]
trainY = tr_df.values[:, -1]

# Test
testX = ts_df.values[:, :-1]
testY = ts_df.values[:, -1]

### Convertir les Y en tensor
Les X las convertimos despues de obtener la imagen del Path en el Dataframe.

In [111]:
import torch

trainY = torch.tensor(trainY.astype(int))
testY = torch.tensor(testY.astype(int))

## Clase Dataset
Al obtener el dataset devolverá la imagen como tensor, que será la X y los resultados Y.

### **Normalizar las imagenes**:
Las imágenes están contenidas dentro de X en forma de Path por un lado y el image name por otro. Para ello tenemos que convertir los paths a una imagen y normalizar los datos para que sean entre 0 y 1 en vez de 0 a 224.

In [112]:
from torch.utils.data import Dataset
import torchvision.transforms.functional as transform
from torchvision.io import read_image
from pathlib import Path
import PIL

class myDataset(Dataset):
    def __init__(self, X, Y):
        self.image_path = X[:, 0]
        #self.image_name = X[:, 1]
        self.Y = Y
        
    def __len__(self):
        return len(self.Y)*4

    def __getitem__(self, idx):
        image = read_image(str(self.image_path[int(idx/4)]))/255.0 # Convertir a tensor y normalizar
        return self.__rotate__(image, idx%4), self.Y[int(idx/4)] # Lo devuelves directamente como una imagen
    
    def __rotate__(self, PIL_image, quarter):
        return transform.rotate(PIL_image, 90*quarter) # Girar imagen

#### Crear el propi dataset
Passar al train_dataloader i test_dataloader un objecte dataset, nosaltres hem de crear aquet dataset extenent de la clase Dataset.

In [113]:
from torch.utils.data import DataLoader

train_dataset = myDataset(trainX, trainY)
test_dataset = myDataset(testX, testY)

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)

### Crear la red neuronal
1. Crear el dispositivo
2. Definir la clase *Module* con la función forward
    - Convulacional (como es en color, cada imagen se multiplica x 3)
    - Capas normales 
3. Crear el modelo y pasarlo a la GPU

In [114]:
from torch import nn

# Ya están implementadas las clases de las capas para hacer el forward
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            # 3 imatges de 224 x 224
            nn.Conv2d(3, 96, (11, 11), stride=4), # Entrada, salida, filtro, filtro cada... 53x53x96 mida - filtre /stride
            nn.MaxPool2d((3, 3)), # 17x17x96

            nn.Conv2d(96, 256, (3, 3), padding=4), #15x15x256
            nn.MaxPool2d((3, 3), stride=2), # 5x5x256

            nn.Conv2d(256, 256, (2, 2)), # 
            nn.MaxPool2d((3, 3), stride=2),
            
            nn.Flatten(),

            nn.Dropout2d(p=0.5, inplace=True), # Para que no haya sobreentrenamiento

            nn.Linear(9216, 3000), 
            nn.ReLU(),
            nn.Linear(3000, 300),
            nn.ReLU(),
            nn.Linear(300, 30),
            nn.ReLU(),
            nn.Linear(30, 2),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x) # Crea las capas
        return logits
    
# Device
if torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

model = NeuralNetwork().to(device) # Otiene los valores predichos

### Train y test
Definimos las funciones para train y test.

In [115]:
import torch
from sklearn.metrics import confusion_matrix

batch_size=64

def train_loop(train_dataloader, model, loss_fn, optimizer):
    size = len(train_dataloader.dataset)

    for batch, (X, Y) in enumerate(train_dataloader):
        X = X.to(device) 
        Y = Y.to(device) 
        
        pred = model(X) # Forward, ya ha calculado todos los gradientes
        loss = loss_fn(pred, Y) # Crear la función de costo: error

        loss.backward() # Le pasa el error al gradiente
        optimizer.step() # Actualiza los valores
        optimizer.zero_grad() # Pone el gradiente a 0

def test_loop(test_dataloader, model, loss_fn):
    size = len(test_dataloader.dataset)
    num_batches = len(test_dataloader)
    
    test_loss, correct = 0, 0

    # No calcula el gradiente automaticamente
    with torch.no_grad():
        for X, Y in test_dataloader:
            
            X = X.to(device)
            Y = Y.to(device)
            
            pred = model(X) # Forward
            test_loss += loss_fn(pred, Y).item() # Error
            predicted_labels = pred.argmax(1) 
            correct += (predicted_labels == Y).type(torch.float).sum().item() # Accuracy
            

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

    # Confusion matrix
    Y = Y.cpu().numpy() # Tensors to arrays
    predicted_labels = predicted_labels.cpu().numpy()
    cm = confusion_matrix(Y, predicted_labels) # Confusion matrix
    print(f"Confusion Matrix:\n{cm}") # TP FP, TN FN

    TP = cm[0, 0]  # True positive
    TN = cm[1, 1]  # True negative
    FP = cm[0, 1]  # False positive
    FN = cm[1, 0]  # False negative
    specificity = (TN / (TN + FP))*100
    sensitivity = (TP / (TP + FN))*100
    print(f"Sensitivity: {sensitivity:.4f}")
    print(f"Specificity: {specificity:.4f}\n")


### Llamar a train y test
En cada epoca hacer un train y un test

In [116]:
import random

learning_rate = 0.0001
loss_fn = nn.CrossEntropyLoss()
model = NeuralNetwork().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

torch.save(model.state_dict(), f'./models/model{random.randint(0,10000)}.pth')

Epoch 1
-------------------------------


/Users/cynthia/anaconda3/envs/NeuralNetwork/lib/python3.11/site-packages/torch/nn/functional.py:1347: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Test Error: 
 Accuracy: 88.2%, Avg loss: 0.274932 

Confusion Matrix:
[[15  1]
 [ 0 16]]

Sensitivity: 100.0000
Specificity: 94.1176
Epoch 2
-------------------------------


/Users/cynthia/anaconda3/envs/NeuralNetwork/lib/python3.11/site-packages/torch/nn/functional.py:1347: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Test Error: 
 Accuracy: 89.0%, Avg loss: 0.249448 

Confusion Matrix:
[[16  3]
 [ 1 12]]

Sensitivity: 94.1176
Specificity: 80.0000
Epoch 3
-------------------------------


/Users/cynthia/anaconda3/envs/NeuralNetwork/lib/python3.11/site-packages/torch/nn/functional.py:1347: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


KeyboardInterrupt: 